<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Dylan/Feature_Engineering_from_Raw_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sys
from numpy.ma.core import ceil, floor
from more_itertools import sliced

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/

210815_0__20220213T070259RAW_stimulus_data.csv
210815_0_allStacks_mc_mix1_syt_rawExtracts


## Functions
This section contains functions that can be used on datasets for feature engineering

In [74]:
raw_df = pd.read_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0__20220213T070259RAW_stimulus_data.csv', index_col=['stack','label','roi','frame']).drop(columns='Unnamed: 0')
raw_df

filename  ...     stim3
stack label roi frame                                                     ...          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.110063
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.030790
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.122134
...                                                                  ...  ...       ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.089182
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.019588
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000

[826950 rows x 5 columns]

# REMOVE THE OUTLIER ROI

In [75]:
#access the 11th roi in the first stack and drop all frames for that roi
raw_df = raw_df.drop(raw_df.loc[1,:,11].index)

In [76]:
raw_df

filename  ...     stim3
stack label roi frame                                                     ...          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.110063
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.030790
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.122134
...                                                                  ...  ...       ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.089182
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.019588
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000

[821437 rows x 5 columns]

## Train Test Split on Stacks

In [ ]:
def trainTestSplit(df, splt = [.7,.2,.1]):
  # splits on roi, rois accross stacks will have the same group (important if stacks are from the same video)
  # split is the train test valuidation split (proportions given in that order)
  if sum(splt) < 0.9999 or sum(splt) > 1.0001:
    sys.exit('Splt must add to 1')
  
  num_rois = len(df.index.unique(level='roi'))

  tr_ind = int(ceil(num_rois*splt[0]))
  ts_ind = int(ceil(num_rois*splt[1])) + tr_ind

  ar = np.arange(num_rois)
  np.random.shuffle(ar)
  ar = ar+1 # roi id's start from 1

  train = ar[:tr_ind]
  test = ar[tr_ind:ts_ind]
  val = ar[ts_ind:]

  for i in train:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'train'

  for i in test:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'test'

  for i in val:
    df.loc[pd.IndexSlice[:, :, i],'training'] = 'validate'

  return(df)

In [ ]:
# df_temp = trainTestSplit(raw_df,splt=[.5,.3,.3]) # should give error
df_temp = trainTestSplit(raw_df,splt=[.5,.3,.2])
df_temp.sample(15)

filename  ...  training
stack label roi frame                                                     ...          
2     t5    12  145    210815_0_2_stackRaw_mc_mix1_syt_result_2022021...  ...     train
5     tm4   9   483    210815_0_5_stackRaw_mc_mix1_syt_result_2022021...  ...     train
            8   2079   210815_0_5_stackRaw_mc_mix1_syt_result_2022021...  ...      test
4     tm2   11  2410   210815_0_4_stackRaw_mc_mix1_syt_result_2022021...  ...     train
2     tm4   15  1733   210815_0_2_stackRaw_mc_mix1_syt_result_2022021...  ...      test
7     tm2   2   3008   210815_0_7_stackRaw_mc_mix1_syt_result_2022021...  ...  validate
1     tm9   6   3182   210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...     train
7     tm4   8   3      210815_0_7_stackRaw_mc_mix1_syt_result_2022021...  ...      test
1     tm2   14  498    210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  validate
8     tm4   9   2057   210815_0_8_stackRaw_mc_mix1_syt_result_2022021...  ...     train
      t5    12  1408   210815_0_8_stackRaw_mc_mix1_syt_result_2022021...  ...     train
1     tm4   15  3680   210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...      test
6     t5    12  289    210815_0_6_stackRaw_mc_mix1_syt_result_2022021...  ...     train
10    tm4   3   3152   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...     train
8     tm2   1   2925   210815_0_8_stackRaw_mc_mix1_syt_result_2022021...  ...      test

[15 rows x 6 columns]

## Combine Frames into Temporal Chunks

In [ ]:
# select frame size
temporal_period_length = 200

In [ ]:

def groupFrames(df, period):
  # inputs are data frame and temporal period length
  # df needs to be sorted correctly, the indexing should take care of that

  # check that all rois are the same size
  # if this is a problem we can write a more computationally heavy workthrough
  if (raw_df.groupby(['stack','roi']).count()['filename'].max() != raw_df.groupby(['stack','roi']).count()['filename'].min()):
    sys.exit('ROIs need to be the same length')

  num_stacks = len(df.index.unique(level='stack'))
  num_roi = len(df.index.unique(level='roi'))

  # get first label
  lbl_1 = raw_df.head(1).index.values[0][1]
  # use it to find the number of frames in a roi
  num_frames = raw_df.loc[(1,lbl_1,1),'filename'].count()
  # how many groups will ther be in each roi?
  num_pds = int(floor(num_frames/period))
  # how many left over
  remainder_pds = num_frames%period

  # iterate a column for one roi
  lst = [[li + 1] * period for li in range(0,num_pds)]
  lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
  lst = lst + [lst[-1] + 1] * remainder_pds
  lst

  new_col = np.array(lst * num_stacks * num_roi)
  
  df['frame_group'] = new_col

  return(df)

# Generate stim_response features

In [82]:
from operator import setitem
def stimResponse(df, stim, response, zero_point = 0):
  """
  Takes a df, the name of the col holding the stimulus, the name of the col holding the response, and the value to use as the zero point

  Returns the df with the relationship type and the relation value added

  A different function will calculate the relationship between time periods
  """


  # takes a stim column and response column and returns two new feature column
  # PR is Positive Response
  # NR is Negative Response
  # PS is Positive Stimulus
  # NS is Negative Stimulus
  df_in = df.copy()
  #
  PR = df_in[response] > zero_point
  NR = df_in[response] <= zero_point
  PS = df_in[stim] > zero_point
  NS = df_in[stim] <= zero_point

  PR = PR.rename('PR')
  NR = NR.rename('NR')
  PS = PS.rename('PS')
  NS = NS.rename('NS')

  # combine into df
  df = pd.concat([df_in[stim], df_in[response], PR,NR,PS,NS], axis=1)
  #set the data types back to boolean
  df = df.astype({'PR': bool,
                'PS': bool,
                'NR': bool,
                'PR': bool})

  # get min and max values for normalizing
  S_min = df[stim].min()
  S_max = df[stim].max()
  R_min = df[response].min()
  R_max = df[response].max()

  #add a column normalizing the stim and response
  df['stim_norm'] = df[stim].apply(lambda x: x/S_max if x>0 else x/S_min)
  df['resp_norm'] = df[response].apply(lambda x: x/R_max if x>0 else x/R_min)


  # multiply stim and resp
  df['relation'] = df['stim_norm'] * df['resp_norm']

  #Set a new column with the category each row's relationship falls under
  df.loc[(df['PR'] & df['PS']),'relation_type'] = 'PRPS'
  df.loc[(df['NR'] & df['NS']),'relation_type'] = 'NRNS'
  df.loc[(df['PR'] & df['NS']),'relation_type'] = 'PRNS'
  df.loc[(df['NR'] & df['PS']),'relation_type'] = 'NRPS'

  #UNCOMMENT this to create the len-4 matrix for the current time period directly

  # #one-hot encode the four categories
  # df = pd.get_dummies(df, columns=['relation_type'], prefix = stim)

  # #get a list of the columns created
  # dummy_cols = [col for col in df.columns if 'relation_type_' in col]

  # #multiply the dummy columns by the relation value to distribute the value to the appropriate column
  # for col in dummy_cols:
  #   df[col] = df[col] * df['relation']
  
  df_in['relation_type_' + stim] = df['relation_type']
  df_in['relation_' + stim] = df['relation']
  return(df_in)

In [202]:
#pass in the raw df, the name of the stimulus we want to use, the response column, and the zero-value (in this case zero)
df = stimResponse(raw_df, 'stim1', 'resp', 0)

In [203]:
df.head()

filename  ...  relation_stim1
stack label roi frame                                                     ...                
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...        0.044529
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...        0.023493
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...        0.038280
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...        0.020845
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...        0.134750

[5 rows x 7 columns]

In [204]:
def get_future_stim_and_relationship(df_in, stim, time_window): 
  """
  takes a df, the stimulus name we're comparing, and the time window we're looking to in the future (in frames)

  returns a new df with two extra columns containing the relationship type and the value at the future time period
  """

  #make copy of the df to avoid editing the one in memory
  df = df_in.copy()

  #procedurally generate the column names we need to reference
  current_relation_colname = 'relation_'+stim
  current_relation_type_colname = 'relation_type_'+stim

  #and the column names we're creating
  future_relation_colname = 'relation_'+stim+'_+'+str(time_window)
  future_relation_type_colname = 'relation_type_'+stim+'_+'+str(time_window)

  #reset the index so that we can join the new columns later
  df = df.reset_index()


  # for each of the new columns, do four steps

  #get the existing array of relation values or relation types

  #cut off the first x values, where x is the time window we want to look in the future (done by indexing [time_window:])
        #this will drop the time x values in the future to line up with the current time

  #pad the end of the array with zeros by the same number of values so that the lengths still match (done by concatenating with np.zeros(time_window))

  #set the new array as the new 'future' column
  df[future_relation_colname] = pd.concat([df[current_relation_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)
  
  #repeat for the relation type
  df[future_relation_type_colname] = pd.concat([df[current_relation_type_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)

  #need to stop the end of one stack from referencing the beginning of another

  #get the index of any row where the frame # is high enough that x-frames in the future would run into another stack
  #by checking that the frame number is greater than the max frame number - (time window-1) (corrects for index number)
  end_of_stack_index = df.loc[df['frame'] >= max(df['frame'])-(time_window-1)].index

  #use .loc to set the two new columns to None for those rows
  df.loc[end_of_stack_index, future_relation_colname] = None
  df.loc[end_of_stack_index, future_relation_type_colname] = None

  #set the index back to the way it was
  df = df.set_index(['stack','label','roi','frame'])

  return df

In [205]:
#check that the values match the records 10 in the future and that the last 10 rows in each stack are null for future values
df = get_future_stim_and_relationship(df, 'stim1', 10)
df.tail(30)

filename  ...  relation_type_stim1_+10
stack label roi frame                                                     ...                         
10    tm4   15  5483   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5484   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5485   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5486   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5487   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     NRNS
                5488   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5489   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5490   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5491   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5492   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5493   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5494   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5495   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5496   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5497   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5498   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRPS
                5499   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5500   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5501   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5502   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     PRNS
                5503   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5504   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5505   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5506   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5507   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...                     None

[30 rows x 9 columns]

In [206]:
#with this approach, we need to drop the last x frames from each stack, so dropna
df = df.dropna()

In [207]:
#make a new column with the combo of current relation type and future relation type
df['full_relation_type'] = df['relation_type_stim1'] + '-' + df['relation_type_stim1_+10']
df.head()

filename  ...  full_relation_type
stack label roi frame                                                     ...                    
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...           NRNS-PRNS
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...           PRNS-PRPS
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...           PRNS-PRPS
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...           PRNS-PRPS
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...           PRPS-PRPS

[5 rows x 10 columns]

In [208]:
#one-hot encode the overall relationship, then multiply the new columns by the product of the relation values

df = pd.get_dummies(df, columns=['full_relation_type'], prefix = 'relation_stim1')

#get a list of the columns created
dummy_cols = [col for col in df.columns if 'relation_stim1' in col]

#multiply the dummy columns by the relation value to distribute the value to the appropriate column
for col in dummy_cols:
  df[col] = df[col] * (df['relation_stim1']*df['relation_stim1_+10'])

df.head(10) 

filename  ...  relation_stim1_PRPS-PRPS
stack label roi frame                                                     ...                          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              1.962413e-08
                5      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              4.142531e-09
                6      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              6.078521e-10
                7      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              2.068366e-12
                8      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                9      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00

[10 rows x 25 columns]

In [210]:
df.to_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0_20220213T070259_sparse_stim1_draft.csv')

In [209]:
df

filename  ...  relation_stim1_PRPS-PRPS
stack label roi frame                                                     ...                          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              0.000000e+00
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...              1.962413e-08
...                                                                  ...  ...                       ...
10    tm4   15  5498   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...              0.000000e+00
                5499   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...              0.000000e+00
                5500   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...              0.000000e+00
                5501   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...              0.000000e+00
                5502   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...              0.000000e+00

[819947 rows x 25 columns]